# Spark Environment Options and Dependencies

This notebook includes examples of how to connect and configure Spark to run in a local environment and to manage the set of dependencies and options required to read/write data to an S3 compatible object storage.

### Environment Notes
* By convention, credentials and secrets are injected into the runtime as environment variables.
  - BASH environment variables are usually named in all caps.
  - Python variables (generally) follow [pep8](https://www.python.org/dev/peps/pep-0008/)
* AWS/S3 variables are prefixed using `OBJECTS_*`

In [1]:
System.setProperty(SDKGlobalConfiguration.DISABLE_CERT_CHECKING_SYSTEM_PROPERTY, "true")

lazy val spark = SparkSession.builder().appName("MinIOTest").master("local[*]").getOrCreate()

val s3accessKeyAws = "minio"
val s3secretKeyAws = "miniosecret"
val connectionTimeOut = "600000"
val s3endPointLoc: String = "http://127.0.0.1:9000"

val sourceBucket: String = "playground"

spark.sparkContext.hadoopConfiguration.set("fs.s3a.endpoint", s3endPointLoc)
spark.sparkContext.hadoopConfiguration.set("fs.s3a.access.key", s3accessKeyAws)
spark.sparkContext.hadoopConfiguration.set("fs.s3a.secret.key", s3secretKeyAws)
spark.sparkContext.hadoopConfiguration.set("fs.s3a.connection.timeout", connectionTimeOut)

spark.sparkContext.hadoopConfiguration.set("spark.sql.debug.maxToStringFields", "100")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.path.style.access", "true")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.connection.ssl.enabled", "true")

SyntaxError: invalid syntax (4251094180.py, line 3)

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
 
spark.sparkContext._jsc\
     .hadoopConfiguration().set("fs.s3a.access.key", "minio")
spark.sparkContext._jsc\
     .hadoopConfiguration().set("fs.s3a.secret.key", "miniosecret")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.endpoint", "http://127.0.0.1:9000")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.connection.timeout", "600000")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("spark.sql.debug.maxToStringFields", "100")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
 
spark.sparkContext._jsc\
     .hadoopConfiguration().set("fs.s3a.access.key", "minio")
spark.sparkContext._jsc\
     .hadoopConfiguration().set("fs.s3a.secret.key", "miniosecret")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("spark.hadoop.fs.s3a.path.style.access", "true")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.multipart.size", "104857600")

In [5]:
ratings = spark.read\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .csv("s3://playground/iris.csv")
ratings.registerTempTable("ratings")

Py4JJavaError: An error occurred while calling o35.csv.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:752)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:750)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(Data

In [1]:
import os, sys, posixpath

# User Options
USER_NAMESPACE = os.environ.get('HOSTNAME')

# Spark Options
SPARK_DRIVER_RAM = '3g'
SPARK_DRIVER_MAXRESULTSIZE = '4096m'
SPARK_WORKER_RAM = '6g'

# Access credentials for Object Storage
OBJECT_STORAGE_URL = os.environ.get('OBJECTS_ENDPOINT')
OBJECT_STORAGE_ACCESSID = os.environ.get('OBJECTS_ACCESSID')
OBJECT_STORAGE_SECRET = os.environ.get('OBJECTS_SECRET')
OBJECTS_PLAYGROUND_BUCKET = 'playground'
OBJECTS_USER_PREFIX = USER_NAMESPACE
OBJECT_CONTAINER = 'minio'
STORAGEURL_PLAYGROUND = 'playground'

# Spark Runtime Options
os.environ['PYSPARK_PYTHON'] = 'python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3'

Spark runtime dependencies. These options control the `jar` and package files which are part of the Spark runtime. This list provides support for working with AWS specific products and APIs, in addition to an updated set required to work with Hadoop 3. **The `JAR_OPTIONS` variable should remain unchanged. For additional runtime dependencies, add them to the `PACKAGE_OPTIONS` array.**

_If you modify `PACKAGE_OPTIONS`, you will also need to add it to the `PYSPARK_SUBMIT_ARGS` string._

In [2]:
# Manage PySpark Runtime Options
PACKAGE_OPTIONS = '--packages %s ' % ','.join((        
        # 'org.apache.spark:spark-avro_2.12:2.4.4',
    ))

JAR_OPTIONS = '--jars %s ' % ','.join((
        '/home/jovyan/work/jars/joda-time-2.9.9.jar',
        '/home/jovyan/work/jars/httpclient-4.5.3.jar',
        '/home/jovyan/work/jars/aws-java-sdk-s3-1.11.534.jar',
        '/home/jovyan/work/jars/aws-java-sdk-kms-1.11.534.jar',
        '/home/jovyan/work/jars/aws-java-sdk-dynamodb-1.11.534.jar',
        '/home/jovyan/work/jars/aws-java-sdk-core-1.11.534.jar',
        '/home/jovyan/work/jars/aws-java-sdk-1.11.534.jar',
        '/home/jovyan/work/jars/hadoop-aws-3.1.2.jar',
        '/home/jovyan/work/jars/slf4j-api-1.7.29.jar',
        '/home/jovyan/work/jars/slf4j-log4j12-1.7.29.jar',
    ))

os.environ['PYSPARK_SUBMIT_ARGS'] = JAR_OPTIONS + ' pyspark-shell'
os.environ.get('PYSPARK_SUBMIT_ARGS')

'--jars /home/jovyan/work/jars/joda-time-2.9.9.jar,/home/jovyan/work/jars/httpclient-4.5.3.jar,/home/jovyan/work/jars/aws-java-sdk-s3-1.11.534.jar,/home/jovyan/work/jars/aws-java-sdk-kms-1.11.534.jar,/home/jovyan/work/jars/aws-java-sdk-dynamodb-1.11.534.jar,/home/jovyan/work/jars/aws-java-sdk-core-1.11.534.jar,/home/jovyan/work/jars/aws-java-sdk-1.11.534.jar,/home/jovyan/work/jars/hadoop-aws-3.1.2.jar,/home/jovyan/work/jars/slf4j-api-1.7.29.jar,/home/jovyan/work/jars/slf4j-log4j12-1.7.29.jar  pyspark-shell'

In [3]:
import pyspark

conf = pyspark.SparkConf()

# Memory
conf.set('spark.driver.memory', SPARK_DRIVER_RAM)
conf.set("spark.executor.memory", SPARK_WORKER_RAM)
conf.set('spark.driver.maxResultSize', SPARK_DRIVER_MAXRESULTSIZE)

#### Spark Driver Hostname and Port
* For programs launched from Jupyter, the Jupyter instance will be the Spark driver.
* Executor pods need to be able to communicate with the driver.

In [4]:
# The DNS alias for the Spark driver. Required by executors to report status.
conf.set("spark.driver.host", os.environ.get('HOSTNAME'))

# Port which the Spark shell should bind to and to which executors will report progress
conf.set("spark.driver.port", "20020")

#### Object Storage Options

* To read/write data from object storage (AWS S3 or MinIO), you need to configure the `S3AFileSystem`.
* Object storage has an endpoint, access ID, and secret. These credentials have been injected into the environment.
* The `OBJECTS_ENDPOINT` is only used for self-hosted environments. For connecting to Amazon's S3, you do not need to set a specific endpoint.

In [5]:
# Configure S3 Object Storage as filesystem, pass MinIO credentials
conf.set("spark.hadoop.fs.s3a.endpoint", OBJECT_STORAGE_URL) \
    .set("spark.hadoop.fs.s3a.access.key", OBJECT_STORAGE_ACCESSID) \
    .set("spark.hadoop.fs.s3a.secret.key", OBJECT_STORAGE_SECRET) \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# Configure Avro options
conf.set('avro.mapred.ignore.inputs.without.extension', 'true')

#### Initialize the Spark Context and Spark Session

In [6]:
# Initialize spark context, create executors
conf.setAppName('iotest')
sc = pyspark.SparkContext(conf=conf)

# Initialize Spark Session
from pyspark.sql import SparkSession
spark = SparkSession(sc)

#### Check that Spark Session is Active

In [7]:
# Create a distributed data set to test to the session
t = sc.parallelize(range(10))

# Calculate the approximate sum of values in the dataset
r = t.sumApprox(3)
print('Approximate sum: %s' % r)

Approximate sum: 45.0


## MinIO Storage Initialization

In [8]:
# Create playground and opioids buckets in Amazon S3
import boto3
from botocore.client import ClientError as AWSClientError

s3 = boto3.client('s3',
    endpoint_url=OBJECT_STORAGE_URL,
    aws_access_key_id=OBJECT_STORAGE_ACCESSID,
    aws_secret_access_key=OBJECT_STORAGE_SECRET,
)

print('URL: %s\nAccess ID: %s\nSecret Key: %s' 
      % (OBJECT_STORAGE_URL, OBJECT_STORAGE_ACCESSID, OBJECT_STORAGE_SECRET))

URL: http://minio:9000
Access ID: minio
Secret Key: miniosecret


In [9]:
for b in (OBJECT_CONTAINER, STORAGEURL_PLAYGROUND):
    print('Inspect bucket structure for %s' % b)
    
    try: s3.head_bucket(Bucket=b)
    except AWSClientError as err:
        print('Bucket %s not present in object store.' % b)
        s3.create_bucket(Bucket=b)
        print('Bucket %s created successfully.' % b)

Inspect bucket structure for minio
Bucket minio not present in object store.


ClientError: An error occurred (AllAccessDisabled) when calling the CreateBucket operation: All access to this resource has been disabled.

### Write Data to MinIO (via Spark)

In [10]:
# Object Output URL
OBJECTURL_TEST = 's3://playground/iris.csv' % (OBJECTS_PLAYGROUND_BUCKET, OBJECTS_USER_PREFIX)
print(OBJECTURL_TEST)

TypeError: not all arguments converted during string formatting

In [ ]:
# Write a two column table to MinIO as CSV
rdd = sc.parallelize([('Mario', 'Red'), ('Luigi', 'Green'), ('Princess', 'Pink')])
rdd.toDF(['name', 'color']).write.csv(OBJECTURL_TEST, header=True)

In [ ]:
# Read the data back from MinIO
gnames_df = spark.read.format('csv').option('header', True) \
    .load(OBJECTURL_TEST)
gnames_df.show()

<div id="parent">
<div id="popup" style="display: none">description text here</div>
</div>

<script>
var e = document.getElementById('parent');
e.onmouseover = function() {
  document.getElementById('popup').style.display = 'block';
}
e.onmouseout = function() {
  document.getElementById('popup').style.display = 'none';
}
</script>

### Managing IO to Object Storage (using Python)
Spark is able to read/write data to object storage by specifying the file path details in the URL. General form:

* Structure: `s3a://{{ bucket }}/{{ object-key-path }}`. Example: `s3a://oak-tree.tech/hello-test.txt`

For Python code, you will need to read and write data utilizing the Python `boto3` library.

In [ ]:
# Example read/write to S3 object storage using Python. Uses the credentials configured earlier in the notebook.

# Configure client with credentials
# Utilizes the S3 boto client session. See https://boto3.amazonaws.com/v1/documentation/api/latest/reference/core/session.html
# for reference and details.
import boto3
s3 = boto3.client('s3',
    endpoint_url=OBJECT_STORAGE_URL,
    aws_access_key_id=OBJECT_STORAGE_ACCESSID,
    aws_secret_access_key=OBJECT_STORAGE_SECRET,
)

print(OBJECT_STORAGE_URL)

# Use requests (Python http client) to retrieve remote data
import requests
from io import StringIO, BytesIO

In [ ]:
#
# Choose EITHER this dataset...
#
# Small(ish) Example (3MB)
url = 'https://data.cdc.gov/api/views/p56q-jrxg/rows.csv?accessType=DOWNLOAD&bom=true&format=true'
filekey = '%s/opioids.mortality.2004-2016' % OBJECTS_USER_PREFIX

In [ ]:
print('Bucket: %s. File key: %s' % (OBJECTS_PLAYGROUND_BUCKET, filekey))

In [ ]:
# Retrieve data and write to Amazon S3 using Python, utilizes BytesIO to provide a file stream
# to the upload_obj method
r0 = requests.get(url, allow_redirects=True)
r1 = s3.upload_fileobj(BytesIO(r0.content), OBJECTS_PLAYGROUND_BUCKET, filekey)

In [ ]:
# Verify that the file was read read into object storage. Results of method call will be dict (hashmap)
for o in s3.list_objects_v2(Bucket=OBJECTS_PLAYGROUND_BUCKET, Prefix=OBJECTS_USER_PREFIX).get('Contents', []):
    
    # Individual objects in the container will also be represented as dict (hashmap)
    print(o.get('Key'))

In [ ]:
# Spark output URL (uses the Amazon s3a file system as output)
# See https://www.redhat.com/en/blog/anatomy-s3a-filesystem-client for a few details
# and benefits of S3a.

# Read as text
spark_in = 's3a://%s/%s' % (OBJECTS_PLAYGROUND_BUCKET, filekey)
print('Retrieve data from %s using Spark' % spark_in)
df = spark.read.text(spark_in)
df.show()

In [ ]:
# Read as structured CSV
df = spark.read.format('csv') \
  .option("inferSchema", True) \
  .option("header", True) \
  .option('sep', ',') \
  .load(spark_in)

df.show()

## Exercise: Stage DOT Flight Data to MinIO
Prior to being able to analyze data in Spark, if must be staged to a specific location that is accessible to the workers (executors) which will process it. Often this will be to a data lake or storage cluster (such as S3 object storage or HDFS). 

The following files will be utilized throughout the remainder of the course. Write a simple program to download the data and stage it within the MinIO cluster in this enviornment so that Spark is able to consume it from the provided bucket/file keys.

In [ ]:
FLIGHTS_DATAFILES = {
    'web-age/wa2843/airports.json': 'https://oak-tree.tech/documents/103/airports.json',
    'web-age/wa2843/dot.flights.2018.json': 'https://oak-tree.tech/documents/105/dot.flights.2018.json',
    'web-age/wa2843/dot.flights.2017-0102.json': 'https://oak-tree.tech/documents/104/dot.flights.2017-0102.json',
}

In [ ]:
# Exercise: Stage Data to MinIO